# 載入所需套件

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import re
from datetime import datetime,timedelta
from tqdm import tqdm
import time

# 定義鉅亨網外資評等個股清單爬蟲函數

In [2]:
def stock_foreign_rate_crawler():
    dfs=[]
    start_date='2018/01/01'
    end_date='2020/10/01'
    #end_date=datetime.now().strftime('%Y/%m/%d')

    url='https://www.cnyes.com/twstock/board/ratediff.aspx'
    session=requests.session()
    response=session.get(url)
    soup=BeautifulSoup(response.text,'html.parser')

    #先從主頁取出VIEWSTATE和VIEWSTATEGENERATOR值
    VIEWSTATE=soup.select_one('#__VIEWSTATE')['value']
    VIEWSTATEGENERATOR=soup.select_one('#__VIEWSTATEGENERATOR')['value']

    #餵入日期範圍取出總頁數
    form_data={'__VIEWSTATE':VIEWSTATE,
               '__VIEWSTATEGENERATOR':VIEWSTATEGENERATOR,
               'ctl00$ContentPlaceHolder1$tbDT01':start_date,
               'ctl00$ContentPlaceHolder1$tbDT02':end_date}

    response=session.post(url,data=form_data)
    soup=BeautifulSoup(response.text,'html.parser')
    temp_page=soup.select_one('#ctl00_ContentPlaceHolder1_PageHelpSc1_Label1').text
    page=int(re.findall('共 (\d+) 頁',temp_page)[0])

    #進入每頁抓取表格
    for i in tqdm(range(1,page+1)):
        new_form_data=form_data.copy()
        new_form_data['ctl00$ContentPlaceHolder1$PageHelpSc1$hide_nowpage']=str(i)
        response=session.post(url,data=new_form_data)
        df=pd.read_html(response.text)[0]
        dfs.append(df)

    total_df=pd.concat(dfs,ignore_index=True)
    
    return total_df

# 鉅亨網外資評等個股清單爬蟲

In [3]:
total_df=stock_foreign_rate_crawler()
total_df.to_csv('外資評等原始資料.csv',index=False,encoding='utf_8_sig')
total_df.head()

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:34<00:00,  1.14it/s]


,評等日期,公司代碼,券商,原評等,升/降,新評等,財測EPS(年度),舊目標價,新目標價,現價,備註
0,20200929,5269-祥碩,美系券商,加碼,重申,加碼,NaN,NaN,2250.0,737.0,--
1,20200922,2354-鴻準,美系券商,買進,降級,中立,NaN,62.0,55.0,54.5,--
2,20200922,3034-聯詠,亞系券商,中立,升級,買進,NaN,327.0,297.0,185.0,--
3,20200918,6213-聯茂,美系券商,無,首次,買進,NaN,NaN,165.0,143.0,--
4,20200917,3037-欣興,美系券商,買進,重申,買進,NaN,NaN,155.0,38.1,--


# 鉅亨網外資評等個股清單資料前處理

In [4]:
dealer_dict={'高盛':'美系券商',
             '花旗':'美系券商',
             '摩根':'美系券商',
             'JP摩根':'美系券商',
             '美林':'美系券商',
             '雷曼':'美系券商',
             '貝爾斯登':'美系券商',
             'CSFB':'歐系券商',
             '匯豐':'歐系券商',
             'BNP':'歐系券商',
             '里昂':'歐系券商',
             '巴克萊':'歐系券商',
             'ING':'歐系券商',
             '嘉誠':'歐系券商',
             '荷銀':'歐系券商',
             'UBS':'亞系券商',
             '麥格理':'亞系券商',
             '德意志':'亞系券商',
             '野村':'亞系券商',
             '大和':'亞系券商',
             '美系券商':'美系券商',
             '歐系券商':'歐系券商',
             '亞系券商':'亞系券商'}

rate_dict={'優於大盤':'超越市場表現',
           '優於大盤表現':'超越市場表現',
           '優於大盤預期':'超越市場表現',
           '優於市場表現':'超越市場表現',
           '突出表現':'超越市場表現',
           '維持優於大盤':'超越市場表現',
           '維持優於市場表現':'超越市場表現',
           '表現優於同業':'超越市場表現',
           '表現優於大盤':'超越市場表現',
           '表現優於指數':'超越市場表現',
           '表現優盤大盤':'超越市場表現',
           '表現突出':'超越市場表現',
           '表現超越大盤':'超越市場表現',
           '超出大盤表現':'超越市場表現',
           '超越大盤':'超越市場表現',
           '超越大盤表現':'超越市場表現',
           '超越市場表現':'超越市場表現',
           '不如大盤表現':'低於市場表現',
           '低於大盤':'低於市場表現',
           '低於大盤表現':'低於市場表現',
           '低於市場表現':'低於市場表現',
           '列於大盤表現':'低於市場表現',
           '劣於大盤':'低於市場表現',
           '落後大盤':'低於市場表現',
           '落後大盤表現':'低於市場表現',
           '表現不及大盤':'低於市場表現',
           '表現不如大盤':'低於市場表現',
           '表現不如指數':'低於市場表現',
           '表現低於大盤':'低於市場表現',
           '表現劣於大盤':'低於市場表現',
           '表現將低於大盤':'低於市場表現',
           '表現落後大盤':'低於市場表現',
           '表現遜於大盤':'低於市場表現',
           '遜於大盤表現':'低於市場表現',
           '中立':'中立',
           '大盤相符':'中立',
           '大盤表現相符':'中立',
           '持平':'中立',
           '符合大盤':'中立',
           '符合大盤表現':'中立',
           '符合市場表現':'中立',
           '符合類股表現':'中立',
           '等同大盤':'中立',
           '維持符合市場表現':'中立',
           '與大盤一致':'中立',
           '與大盤持平':'中立',
           '與大盤權重一致':'中立',
           '與大盤相仿':'中立',
           '與大盤表現相仿':'中立',
           '與大盤表現相符':'中立',
           '與市場相仿':'中立',
           '表現等同大盤':'中立',
           '表現與大盤相仿':'中立',
           '觀望':'中立',
           '保持持股':'持有',
           '繼續持有':'持有',
           '續抱':'持有',
           '持有':'持有',
           '持股續抱':'持有',
           '調升':'買進',
           '具吸引力':'買進',
           '正向':'買進',
           '正面':'買進',
           '買進':'買進',
           '逢低佈局':'買進',
           '逢低買進':'買進',
           '賣出':'賣出',
           '逢高出脫':'賣出',
           '逢高賣出':'賣出',
           '加碼':'加碼',
           '加碼買進':'加碼',
           '增持':'加碼',
           '減少持股':'減碼',
           '減持':'減碼',
           '減碼':'減碼',
           '強力買進':'強力買進',
           '強烈買進':'強力買進',
           '強烈買進':'強力買進',
           '強力賣出':'強力賣出'}

total_df=pd.read_csv('外資評等原始資料.csv')
total_df.drop(columns=['現價','備註'],inplace=True)
total_df['評等日期']=total_df['評等日期'].astype(str)
total_df=total_df[total_df['評等日期'].str.len()==8]
total_df['評等年份']=total_df['評等日期'].apply(lambda x:x[:4])
total_df['評等日期']=total_df['評等日期'].apply(lambda x:datetime.strftime(datetime.strptime(x,'%Y%m%d'),'%Y/%m/%d'))
total_df['券商']=total_df['券商'].map(dealer_dict)
total_df['股票代號']=total_df['公司代碼'].apply(lambda x:x[:4])
total_df['股票名稱']=total_df['公司代碼'].apply(lambda x:x[5:])
total_df=total_df[~total_df['新評等'].str.contains('-|無')]
total_df['新評等']=total_df['新評等'].map(rate_dict)
total_df.reset_index(drop=True,inplace=True)

In [5]:
total_df.head()

,評等日期,公司代碼,券商,原評等,升/降,新評等,財測EPS(年度),舊目標價,新目標價,評等年份,股票代號,股票名稱
0,2020/09/29,5269-祥碩,美系券商,加碼,重申,加碼,NaN,NaN,2250.0,2020,5269,祥碩
1,2020/09/22,2354-鴻準,美系券商,買進,降級,中立,NaN,62.0,55.0,2020,2354,鴻準
2,2020/09/22,3034-聯詠,亞系券商,中立,升級,買進,NaN,327.0,297.0,2020,3034,聯詠
3,2020/09/18,6213-聯茂,美系券商,無,首次,買進,NaN,NaN,165.0,2020,6213,聯茂
4,2020/09/17,3037-欣興,美系券商,買進,重申,買進,NaN,NaN,155.0,2020,3037,欣興


# 抓取Yahoo Finance個股股價資料

In [6]:
stock_ids=total_df['股票代號'].unique()
stock_tickers=[['{}.TW'.format(e),'{}.TWO'.format(e)] for e in stock_ids]
stock_price_dict={}

for i in tqdm(range(len(stock_tickers))):
    try:
        stock_price_dict[stock_ids[i]]=pdr.DataReader(stock_tickers[i][0],'yahoo',start='2018/01/01',end=datetime.now().strftime('%Y/%m/%d')).resample('D').ffill()
    except:
        stock_price_dict[stock_ids[i]]=pdr.DataReader(stock_tickers[i][1],'yahoo',start='2018/01/01',end=datetime.now().strftime('%Y/%m/%d')).resample('D').ffill()  

100%|████████████████████████████████████████████████████████████████████████████████| 113/113 [05:28<00:00,  3.19s/it]


# 計算個股股價變動百分比+合併資料

In [7]:
dates=total_df['評等日期'].values
ids=total_df['股票代號'].values

for i in tqdm(range(len(total_df))):
    t0=dates[i]
    total_df.loc[i,'收盤價_0']=stock_price_dict[ids[i]].loc[t0,'Adj Close']
    
    for j in range(1,6):
        tj=datetime.strftime(datetime.strptime(t0,'%Y/%m/%d')+timedelta(days=j),'%Y/%m/%d')
        total_df.loc[i,'收盤價_{}'.format(j)]=stock_price_dict[ids[i]].loc[tj,'Adj Close']
        total_df.loc[i,'百分比_{}'.format(j)]=(total_df.loc[i,'收盤價_{}'.format(j)]/total_df.loc[i,'收盤價_0']-1)*100

100%|████████████████████████████████████████████████████████████████████████████████| 675/675 [00:10<00:00, 61.45it/s]


In [8]:
total_df.head()

,評等日期,公司代碼,券商,原評等,升/降,新評等,財測EPS(年度),舊目標價,新目標價,評等年份,...,收盤價_1,百分比_1,收盤價_2,百分比_2,收盤價_3,百分比_3,收盤價_4,百分比_4,收盤價_5,百分比_5
0,2020/09/29,5269-祥碩,美系券商,加碼,重申,加碼,NaN,NaN,2250.0,2020,...,1455.000000,4.301075,1455.000000,4.301075,1455.000000,4.301075,1455.000000,4.301075,1455.000000,4.301075
1,2020/09/22,2354-鴻準,美系券商,買進,降級,中立,NaN,62.0,55.0,2020,...,51.400002,-1.153843,50.299999,-3.269232,50.400002,-3.076920,50.400002,-3.076920,50.400002,-3.076920
2,2020/09/22,3034-聯詠,亞系券商,中立,升級,買進,NaN,327.0,297.0,2020,...,259.500000,0.193050,258.000000,-0.386100,243.500000,-5.984556,243.500000,-5.984556,243.500000,-5.984556
3,2020/09/18,6213-聯茂,美系券商,無,首次,買進,NaN,NaN,165.0,2020,...,133.500000,0.000000,133.500000,0.000000,128.500000,-3.745318,129.500000,-2.996255,129.000000,-3.370787
4,2020/09/17,3037-欣興,美系券商,買進,重申,買進,NaN,NaN,155.0,2020,...,78.199997,-2.493766,78.199997,-2.493766,78.199997,-2.493766,78.900002,-1.620942,77.400002,-3.491266


In [9]:
total_pos_df=total_df.query('新評等=="買進"|新評等=="超越市場表現"|新評等=="加碼"|新評等=="強力買進"')
total_neg_df=total_df.query('新評等=="賣出"|新評等=="低於市場表現"|新評等=="減碼"|新評等=="強力賣出"')
total_neu_df=total_df.query('新評等=="中立"|新評等=="持有"')

pos_right_cond='|'.join(['百分比_{}>=3'.format(i) for i in range(1,6)])
neg_right_cond='|'.join(['百分比_{}<=-3'.format(i) for i in range(1,6)])

dealer_list=['美系券商','歐系券商','亞系券商']

for dealer in dealer_list:
    dealer_cond='券商=="{}"'.format(dealer)
    pos_df=total_pos_df.query(dealer_cond)
    neg_df=total_neg_df.query(dealer_cond)
    pos_right_df=pos_df.query(pos_right_cond)
    neg_right_df=neg_df.query(neg_right_cond)
    pos_right_rate=len(pos_right_df)/len(pos_df)*100
    neg_right_rate=len(neg_right_df)/len(neg_df)*100
    total_right_rate=(len(pos_right_df)+len(neg_right_df))/(len(pos_df)+len(neg_df))*100
    
    print('{}評等喊多正確率 : {:5.2f}%'.format(dealer,pos_right_rate))
    print('{}評等喊空正確率 : {:5.2f}%'.format(dealer,neg_right_rate))
    print('{}評等多空正確率 : {:5.2f}%'.format(dealer,total_right_rate))
    print('\n')

美系券商評等喊多正確率 : 34.81%
美系券商評等喊空正確率 : 43.64%
美系券商評等多空正確率 : 36.86%


歐系券商評等喊多正確率 : 35.71%
歐系券商評等喊空正確率 : 33.33%
歐系券商評等多空正確率 : 35.29%


亞系券商評等喊多正確率 : 33.71%
亞系券商評等喊空正確率 : 46.81%
亞系券商評等多空正確率 : 36.44%


